Sources:
1. map borders: https://gis-support.pl/baza-wiedzy-2/dane-do-pobrania/granice-administracyjne/
2. deaths info: read from `data/preprocessed/lollipop_chart.csv`, created during initialization

In [1]:
library(rgdal)
library(cartography)
library(stringi)
library(dplyr)
s <- Sys.setlocale("LC_CTYPE","polish")

setwd("./data/maps/")
extractedFiles <- list.files()
fileWithShapesName <- extractedFiles[!is.na(stri_match_first_regex(extractedFiles,".shp"))]
fileWithShapesName <- stri_replace_all_fixed(fileWithShapesName,".shp","")
my_spdf1 <- readOGR( dsn= "." , layer=fileWithShapesName, verbose=FALSE, encoding = 'UTF-8') 
setwd("../..")

Encoding(my_spdf1@data$JPT_NAZWA_) <- "UTF-8"
orderOnMap <- my_spdf1@data$JPT_NAZWA_


df_base <- read.csv("data/preprocessed/lollipop_chart.csv", stringsAsFactors = FALSE, encoding='UTF-8')

Warning message:
"package 'rgdal' was built under R version 4.0.3"
Loading required package: sp

Warning message:
"package 'sp' was built under R version 4.0.3"
rgdal: version: 1.5-18, (SVN revision 1082)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.0.4, released 2020/01/28
Path to GDAL shared files: C:/Users/Filmos/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 6.3.1, February 10th, 2020, [PJ_VERSION: 631]
Path to PROJ shared files: C:/Users/Filmos/Documents/R/win-library/4.0/rgdal/proj
Linking to sp version:1.4-4
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.

Warning message:
"package 'cartography' was built under R version 4.0.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:b

In [2]:
# preparing and sorting data about deaths in the same order as on the map
df <- mutate(df_base, deathsPerMilionCitizens2020 = rok_2020/ilosc_mieszkancow_2020*1000000,
         wojewodztwo = tolower(wojewodztwo)) %>%
  select(wojewodztwo, deathsPerMilionCitizens2020) 
  
df <- df[order(match(df$wojewodztwo, orderOnMap)),]


# making map
my_spdf1@data$dane <- df$deathsPerMilionCitizens2020
breaks <- 1000+150*1:6

png(filename = "charts/death_map.png", width=840, height=840)
par(bg="#00000000")
choroLayer(spdf = my_spdf1, 
           var = "dane",
           breaks = breaks,
           col = c("#c9779c","#b16a8e","#9a5e80","#8c5d8a","#785d92"),
           legend.title.txt = "Total deaths",
           legend.title.cex = 3,
           legend.values.rnd = 1,
           legend.values.cex = 2,
           legend.pos = c(7,49),
           legend.horiz = TRUE,
           border = "black")
title("Total deaths in 2020",cex.main=4)
dev.off()


png 
  2

In [3]:
# preparing and sorting data in the same order as on the map
df <- mutate(df_base, wojewodztwo = tolower(wojewodztwo),
         ilosc_mieszkancow_2020 = round(ilosc_mieszkancow_2020/1000000,1)) %>%
  select(wojewodztwo, ilosc_mieszkancow_2020) 

df <- df[order(match(df$wojewodztwo, orderOnMap)),]


# making plot
my_spdf1@data$dane <- df$ilosc_mieszkancow_2020
breaks <- 1:6

png(filename = "charts/population_map.png", width=840, height=840)
par(bg="#00000000")
choroLayer(spdf = my_spdf1, 
           var = "dane",
         #  col=cols,
           breaks = breaks,
           col = c("#c9779c","#b16a8e","#9a5e80","#8c5d8a","#785d92"),
           legend.title.txt = "Population\n(in mln)",
           legend.title.cex = 3,
           legend.values.rnd = 1,
           legend.values.cex = 2,
           legend.pos = c(9,49),
           legend.horiz = TRUE,
           border = "black")
title("Population",cex.main=4)
dev.off()

# to save the map without background press "zoom", open new window to fullscreen
# and then click with right button and select "save as..."

# adjust text size with *.cex parameters above

png 
  2